In [108]:
from mnist import MNIST
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm, trange
from more_itertools import chunked

In [7]:
mntrain = MNIST('../samples')

img_tr,lbl_tr = mntrain.load_training()
img_ts,lbl_ts = mntrain.load_testing()
img_size = 28 # 28x28 pixels

def display_image(img):
    plt.imshow(np.array(img).reshape(img_size,img_size), cmap=plt.cm.binary)
    plt.gcf().axes[0].set_axis_off()
    plt.show()

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def d_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))
def d_sig(x):
    return x*(1-x)

In [321]:
class Layer:
    def __init__(self, length):
        self.length = length
        self.nodes = np.zeros(length)
        
    def __str__(self):
        return "Layer: length: %d" % (self.length)
    def __repr__(self):
        return self.__str__()
    def __getitem__(self, index):
        return self.nodes[index]
    def __setitem__(self, index, value):
        self.nodes[index].value = value
    def __iter__(self):
        return iter(self.nodes)
    def __len__(self):
        return len(self.nodes)
    def __contains__(self, item):
        return item in self.nodes
    
    def get_values(self):
        return self.nodes
    def set_values(self, values):
        self.nodes = np.array(values)
    values = property(get_values, set_values)

    
class Neurons:
    def __init__(self, input: Layer, output: Layer):
        self.input = input
        self.output = output
        self.weights = np.random.rand(len(output), len(input))*0.2-0.1
        self.biases = np.random.rand(len(output))*0.2-0.1

    def __str__(self):
        return "Layer_Connection: layer1: %s, layer2: %s, weights: %s, biases: %s" % (len(self.input), len(self.output), self.weights.shape, self.biases.shape)
    def __repr__(self):
        return self.__str__()
    def __getitem__(self, index):
        return self.weights[index]
    def __setitem__(self, index, value):
        self.weights[index] = value
    def __iter__(self):
        return iter(self.weights)
    def __len__(self):
        return self.weights.shape[0] * self.weights.shape[1]
    def __contains__(self, layer):
        return layer in (self.input, self.output)


class Neural_Network:
    def __init__(self, input_size, output_size,hidden_size, hidden_layers=1, batch_size=1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        
        self.layers = [Layer(input_size)]
        self.layers.extend([Layer(hidden_size) for _ in range(hidden_layers)])
        self.layers.append(Layer(output_size))
        
        self.neurons = [Neurons(self.layers[i], self.layers[i+1]) for i in range(len(self.layers)-1)][::-1]
    
    def __str__(self):
        return "Neural_Network: input_size: %d, hidden_size: %d, output_size: %d, hidden_layers: %d, layers: %s, connections: %s" % (self.input_size, self.hidden_size, self.output_size, len(self.layers)-2, self.layers, self.neurons)
    def __repr__(self):
        return self.__str__()
        
    def output_vector(self):
        return self.layers[-1].values
    
    def output(self):
        return np.argmax(self.output_vector())
    
    def feed_forward(self, image):
        self.layers[0].values = np.array(image)/256
        neurs = self.neurons[::-1]
        for depth, layer in enumerate(self.layers[1:]):
            layer.values = sigmoid(np.dot(neurs[depth].weights, self.layers[depth]) + neurs[depth].biases)
            

    def back_propagate(self, images, labels, factor):
        label_arrays = [np.zeros(self.output_size) for _ in range(len(labels))]
        for i,label in enumerate(labels):
            label_arrays[i][label] = 1
            
        total_nabla_w = [np.zeros(neuron.weights.shape) for neuron in self.neurons]
        total_nabla_b = [np.zeros(neuron.biases.shape) for neuron in self.neurons]
        for i in range(len(images)):
            nabla_w, nabla_b = self.back_propagate_single(images[i], label_arrays[i])
            for j in range(len(nabla_w)):
                total_nabla_w[j] += nabla_w[j]
                total_nabla_b[j] += nabla_b[j]
            
        for i in range(len(self.neurons)):
            self.neurons[i].weights += factor * total_nabla_w[i]/len(images)
            self.neurons[i].biases += factor * total_nabla_b[i]/len(images)
        
    def back_propagate_single(self, image, label):
        self.feed_forward(image)
        nabla_w = []
        nabla_b = []
        for i in range(len(self.neurons)):
            neuron = self.neurons[i]
            if i == 0: #output layer
                delta = np.array(neuron.output.values - label) * np.array(d_sig(neuron.output.values))
            else:
                delta = np.matmul(self.neurons[i-1].weights.T, delta) * d_sig(neuron.output.values)
            delta_w = np.outer(delta, neuron.input.values)
            nabla_w.append(delta_w)
            nabla_b.append(delta)
        return nabla_w, nabla_b

    
    def test(self, images, labels):
        correct = 0
        for i in trange(len(images)):
            self.feed_forward(images[i])
            if self.output() == labels[i]:
                correct += 1
        return correct/len(images)

              
    def train(self, images, labels, epochs=1, factor=0.1):
        img_lbls = list(zip(images,labels))
        for _ in trange(epochs):
            np.random.shuffle(img_lbls)
            batches = chunked(img_lbls, self.batch_size)
            for batch in tqdm(batches, leave=False, desc="Epoch", total = len(images)//self.batch_size):
                batch_images = [image for image,_ in batch]
                batch_labels = [label for _,label in batch]
                self.back_propagate(batch_images, batch_labels, factor)

In [327]:
nn = Neural_Network(784, 10, 16, 1, 1000)

In [328]:

# nn.feed_forward(img_ts[2])
# for i in nn.layers:
#     print(i.values)
# print(sigmoid(np.matmul(nn.neurons[-1].weights,nn.layers[0]) + nn.neurons[-1].biases))

In [329]:
nn.train(img_tr,lbl_tr, 100, 0.1)

100%|██████████| 100/100 [28:44<00:00, 17.24s/it]


In [330]:
nn.test(img_ts,lbl_ts)

100%|██████████| 10000/10000 [00:02<00:00, 4343.04it/s]


0.0794